# Connecting to Aurora DSQL in a Jupyter notebook

This notebook shows how to use Jupyter to connect to Aurora DSQL using the Aurora DSQL Connector for Python and Psycopg. It
installs required modules, sets up a connection with Aurora DSQL, and shows some basic data usage.

## Requirements

* An Aurora DSQL cluster
* AWS credentials configured in your environment
* Python 3

## Install and import required modules

This notebook uses the Aurora DSQL Connector for Python with Psycopg for database connection, as well as pandas and numexpr for data analysis


In [ ]:
pip install aurora_dsql_python_connector psycopg pandas numexpr

In [ ]:
import pandas as pd
import aurora_dsql_psycopg as dsql
import os

## Download the Amazon root certificate from the official trust store

SSL certificate verification is recommended with Aurora DSQL. The following will download the official Amazon root certificate which will then be used to connect to Aurora DSQL with SSL.

## Connecting to Aurora DSQL

The following demonstrates creation of an Aurora DSQL connection using the Aurora DSQL Connector and Psycopg. Ensure you set the environment variables `CLUSTER_ENDPOINT`, `CLUSTER_USER`, and `REGION`, or alternatively modify the code to hardcode your configuration values.

In [ ]:
cluster_endpoint = os.environ.get("CLUSTER_ENDPOINT", None)
cluster_user = os.environ.get("CLUSTER_USER", None)
region = os.environ.get("REGION", None)
config = {
    'host': cluster_endpoint,
    'region': region,
    'user': cluster_user,
    'sslmode': 'verify-full',
    'sslrootcert': './root.pem'
}

conn = dsql.connect(**config)
conn.autocommit = True
cur = conn.cursor()

### Add sample data to your cluster

In [ ]:
cur.execute("DROP TABLE IF EXISTS owners")
cur.execute("""
    CREATE TABLE IF NOT EXISTS owners(
        id uuid NOT NULL DEFAULT gen_random_uuid(),
        name varchar(30) NOT NULL,
        city varchar(80) NOT NULL,
        telephone varchar(20) DEFAULT NULL,
        PRIMARY KEY (id))
        """)
sample_data = [
    ('John Doe', 'Anytown', '555-555-1999'),
    ('Alice Smith', 'Seattle', '555-555-0123'),
    ('Bob Johnson', 'Portland', '555-555-0456'),
    ('Carol Davis', 'Seattle', '555-555-0789')
]

cur.executemany("INSERT INTO owners(name, city, telephone) VALUES(%s, %s, %s)", sample_data)

### Query and analyze data from Aurora DSQL

In the example below we are selecting all values from the `owners` table and transforming them into a Pandas data frame, though
many other Python data frameworks would work here too.

In [ ]:
cur.execute("SELECT * FROM owners")

# The execute returns a list of tuples:
tuples_list = cur.fetchall()


column_names = ['id','name','city', 'telephone']

df = pd.DataFrame(tuples_list, columns=column_names)

# Analyze the data
print(df)
print(f"Total records: {len(df)}")
print(f"Number of unique cities: {len(df['city'].unique())}")

### Clean up DSQL resources

In [ ]:
cur.execute("DROP TABLE IF EXISTS owners")
cur.close()
conn.close()